In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
c_drone_path = '../../../inner10m/*.wav'
m_drone_path = '../../../20m/*.wav'
f_drone_path = '../../../50m/*.wav'
background_path = '../../data/background/*.wav'

c_drone_files = glob.glob(c_drone_path)
m_drone_files = glob.glob(m_drone_path)
f_drone_files = glob.glob(f_drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
c_drone_raw = load(c_drone_files)
m_drone_raw = load(m_drone_files)
f_drone_raw = load(f_drone_files)
background_raw = load(background_files)

(4464640,)
(2232320,)
(2232320,)
(23317637,)


# Data Processing

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
c_mfcc_drone, c_y_drone = mfcc4(c_drone_raw, 3)
m_mfcc_drone, m_y_drone = mfcc4(m_drone_raw, 2)
f_mfcc_drone, f_y_drone = mfcc4(f_drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(c_mfcc_drone.shape, c_y_drone.shape)
print(m_mfcc_drone.shape, m_y_drone.shape)
print(f_mfcc_drone.shape, f_y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(4464640,)
small end: (16, 9)
(2232320,)
small end: (16, 9)
(2232320,)
small end: (16, 9)
(23317637,)
small end: (16, 15)
small end: (16, 7)
(1089, 16, 16) (1089,)
(544, 16, 16) (544,)
(544, 16, 16) (544,)
(5691, 16, 16) (5691,)


In [8]:
X = np.concatenate((c_mfcc_drone,m_mfcc_drone,f_mfcc_drone, mfcc_background), axis=0)
#X = np.concatenate((mfcc_drone), axis=0)
#X = X.reshape(-1, 16,16,1)
y = np.hstack((c_y_drone, m_y_drone, f_y_drone, y_background))
#y = np.hstack(y_drone)
print(X.shape, y.shape)

(7868, 16, 16) (7868,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 4
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(7868, 4)


In [10]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape, X_test.shape)
print(X_val.shape, y_val.shape)
print(y_train.shape, y_test.shape)

(5035, 16, 16) (1574, 16, 16)
(1259, 16, 16) (1259, 4)
(5035, 4) (1574, 4)


In [12]:
# Save Data
np.save('../../data/X_train_cnn', X_train)
np.save('../../data/X_test_cnn', X_test)
np.save('../../data/X_val_cnn', X_val)
np.save('../../data/y_val_cnn', y_val)
np.save('../../data/y_train_cnn', y_train)
np.save('../../data/y_test_cnn', y_test)

### Until this part

In [13]:
# Load Data
X_train = np.load('../../data/X_train_cnn.npy')
X_test = np.load('../../data/X_test_cnn.npy')
X_val = np.load('../../data/X_val_cnn.npy')
y_val = np.load('../../data/y_val_cnn.npy')
y_train = np.load('../../data/y_train_cnn.npy')
y_test = np.load('../../data/y_test_cnn.npy')

# Experiment 3 - One convolutional layer /w no dropout

##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [14]:
tf.reset_default_graph()

In [15]:
n_mfcc = 16
n_frame = 16
n_classes = 4
n_channels = 1

learning_rate = 0.0002  # 0.005
training_epochs = 500 # 수정해봐

# Layer

In [16]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
# dropout넣어야하나
conv2 = tf.layers.conv2d(inputs=pool1, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=1)
# 여기도
flat = tf.reshape(pool2, [-1, 16*16*1])
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense2, units=4)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
Y_pred = tf.contrib.layers.fully_connected(logits,n_classes,activation_fn = None)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_val2 = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [21]:
# model save
model_path = '../../model/CNN/4_cnn_model'
saver = tf.train.Saver()

# Trainning

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools as it

In [23]:
###########################
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)
with tf.device("/gpu:0"):
    for epoch in range(training_epochs):#training epoch 500 / batch_size 128 --> acc 90%
        avg_cost = 0
        val_avg_cost =0
        total_batch = int(y_train.shape[0] / batch_size)
        for i in range(0, y_train.shape[0], batch_size):
            feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            cost_history = np.append(cost_history,cost)
            avg_cost += c/total_batch 

        y_pred = sess.run(logits, feed_dict={X:X_val2})
        y_pred = sess.run(tf.argmax(y_pred,1))
        y_true = y_val
        
        y_true = sess.run(tf.argmax(y_true,1))
        print(len(y_pred),end=' ')
        print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost), 'val = ','%f' %(accuracy_score(y_true, y_pred)) )
saver.save(sess, model_path)

1259 Epoch: 0001 cost =  0.551835357 val =  0.858618
1259 Epoch: 0002 cost =  0.375906276 val =  0.878475
1259 Epoch: 0003 cost =  0.299791222 val =  0.896743
1259 Epoch: 0004 cost =  0.247997445 val =  0.903892
1259 Epoch: 0005 cost =  0.218807699 val =  0.910246
1259 Epoch: 0006 cost =  0.198095807 val =  0.918189
1259 Epoch: 0007 cost =  0.181707018 val =  0.921366
1259 Epoch: 0008 cost =  0.169363271 val =  0.924543
1259 Epoch: 0009 cost =  0.159218395 val =  0.926132
1259 Epoch: 0010 cost =  0.149393265 val =  0.926926
1259 Epoch: 0011 cost =  0.141300385 val =  0.927720
1259 Epoch: 0012 cost =  0.134222272 val =  0.930898
1259 Epoch: 0013 cost =  0.127086242 val =  0.928515
1259 Epoch: 0014 cost =  0.120366601 val =  0.932486
1259 Epoch: 0015 cost =  0.114238074 val =  0.929309
1259 Epoch: 0016 cost =  0.107887729 val =  0.929309
1259 Epoch: 0017 cost =  0.102105655 val =  0.929309
1259 Epoch: 0018 cost =  0.096623216 val =  0.926926
1259 Epoch: 0019 cost =  0.091913456 val =  0.

1259 Epoch: 0156 cost =  0.000004149 val =  0.941223
1259 Epoch: 0157 cost =  0.000003825 val =  0.941223
1259 Epoch: 0158 cost =  0.000003517 val =  0.941223
1259 Epoch: 0159 cost =  0.000003220 val =  0.940429
1259 Epoch: 0160 cost =  0.000002962 val =  0.940429
1259 Epoch: 0161 cost =  0.000002725 val =  0.941223
1259 Epoch: 0162 cost =  0.000002496 val =  0.941223
1259 Epoch: 0163 cost =  0.000002289 val =  0.940429
1259 Epoch: 0164 cost =  0.000002102 val =  0.942017
1259 Epoch: 0165 cost =  0.000001932 val =  0.941223
1259 Epoch: 0166 cost =  0.000001783 val =  0.941223
1259 Epoch: 0167 cost =  0.000001630 val =  0.942017
1259 Epoch: 0168 cost =  0.000001498 val =  0.942812
1259 Epoch: 0169 cost =  0.000001373 val =  0.942017
1259 Epoch: 0170 cost =  0.000001261 val =  0.942812
1259 Epoch: 0171 cost =  0.000001156 val =  0.942812
1259 Epoch: 0172 cost =  0.000001062 val =  0.942812
1259 Epoch: 0173 cost =  0.000000976 val =  0.943606
1259 Epoch: 0174 cost =  0.000000894 val =  0.

1259 Epoch: 0311 cost =  0.000000465 val =  0.944400
1259 Epoch: 0312 cost =  0.000000439 val =  0.944400
1259 Epoch: 0313 cost =  0.000000414 val =  0.944400
1259 Epoch: 0314 cost =  0.000000391 val =  0.944400
1259 Epoch: 0315 cost =  0.000000369 val =  0.944400
1259 Epoch: 0316 cost =  0.000000348 val =  0.944400
1259 Epoch: 0317 cost =  0.000000328 val =  0.944400
1259 Epoch: 0318 cost =  0.000000310 val =  0.944400
1259 Epoch: 0319 cost =  0.000000292 val =  0.944400
1259 Epoch: 0320 cost =  0.000000276 val =  0.944400
1259 Epoch: 0321 cost =  0.000000260 val =  0.944400
1259 Epoch: 0322 cost =  0.000000246 val =  0.944400
1259 Epoch: 0323 cost =  0.000000233 val =  0.944400
1259 Epoch: 0324 cost =  0.000000219 val =  0.944400
1259 Epoch: 0325 cost =  0.000000207 val =  0.944400
1259 Epoch: 0326 cost =  0.000000195 val =  0.943606
1259 Epoch: 0327 cost =  0.000000184 val =  0.943606
1259 Epoch: 0328 cost =  0.000000173 val =  0.943606
1259 Epoch: 0329 cost =  0.000000163 val =  0.

1259 Epoch: 0466 cost =  0.000000335 val =  0.945989
1259 Epoch: 0467 cost =  0.000000318 val =  0.945989
1259 Epoch: 0468 cost =  0.000000301 val =  0.945195
1259 Epoch: 0469 cost =  0.000000284 val =  0.944400
1259 Epoch: 0470 cost =  0.000000269 val =  0.944400
1259 Epoch: 0471 cost =  0.000000254 val =  0.944400
1259 Epoch: 0472 cost =  0.000000241 val =  0.944400
1259 Epoch: 0473 cost =  0.000000228 val =  0.944400
1259 Epoch: 0474 cost =  0.000000215 val =  0.944400
1259 Epoch: 0475 cost =  0.000000203 val =  0.944400
1259 Epoch: 0476 cost =  0.000000193 val =  0.944400
1259 Epoch: 0477 cost =  0.000000182 val =  0.944400
1259 Epoch: 0478 cost =  0.000000173 val =  0.944400
1259 Epoch: 0479 cost =  0.000000163 val =  0.944400
1259 Epoch: 0480 cost =  0.000000154 val =  0.944400
1259 Epoch: 0481 cost =  0.000000146 val =  0.944400
1259 Epoch: 0482 cost =  0.000000138 val =  0.944400
1259 Epoch: 0483 cost =  0.000000130 val =  0.943606
1259 Epoch: 0484 cost =  0.000000122 val =  0.

'../../model/CNN/4_cnn_model'

## Prediction

In [24]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [25]:
# Print Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.941
Accuracy:  0.9409148665819568
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1118
           1       0.78      0.79      0.78       104
           2       0.71      0.66      0.68       117
           3       0.92      0.92      0.92       235

   micro avg       0.94      0.94      0.94      1574
   macro avg       0.85      0.84      0.84      1574
weighted avg       0.94      0.94      0.94      1574

[[1106    2    4    6]
 [   2   82   18    2]
 [  13   17   77   10]
 [   6    4    9  216]]
